In [67]:
"""
Created on Tuesday 21 Sep 2023
Author: ZAW
"""
##import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

### Read the dataset

In [68]:
df = pd.read_excel('data/list_of_detected_peak_pairs.xlsx',skiprows=2)

In [69]:
df.columns

Index(['No', 'RT (s)', 'Normalized RT (s)', 'm/z_light', 'm/z_heavy',
       'Neutral Mass (Da)', 'Intensity', 'nCharge', 'nTag', 'Unnamed: 9',
       'NovaMT Library No.', 'External Identifier', 'Compound',
       'Mass Error (ppm)', 'RT Error (s)', 'NovaMT Library No..1',
       'External Identifier.1', 'Compound.1', 'Mass Error (ppm).1',
       'RT Error (s).1', 'Zero-reaction', 'One-reaction', 'Two-reaction',
       'Unnamed: 23', 'DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6',
       'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12',
       'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17', 'DR.18',
       'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22', 'DR.23',
       'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25',

### Extract the relevant columns from the dataframe

In [70]:
columns = ['Normalized RT (s)','m/z_light', 'm/z_heavy',
       'Neutral Mass (Da)', 'Intensity','Compound.1','DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6',
       'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12',
       'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17', 'DR.18',
       'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22', 'DR.23',
       'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25', 'No DR.26', 'No DR.27',
       'No DR.28', 'No DR.29']
df_extracted = df[columns]

In [71]:
df.drop(df[df['Compound.1'].isna() | (df['Compound.1'] == '')].index, inplace=True)

In [77]:
len(df['Compound.1'])

101

In [81]:
len(df)

101

In [80]:
df

,No,RT (s),Normalized RT (s),m/z_light,m/z_heavy,Neutral Mass (Da),Intensity,nCharge,nTag,Unnamed: 9,...,No DR.27,No DR.28,No DR.29,QC,QC.1,QC.2,QC.3,QC.4,QC.5,QC.6
19,K-40,123.6,392.5,307.1225,309.1285,59.0373,1152.039064,1,1,Tier 2,...,0.522,0.664,0.746,0.722,0.811,0.907,0.800,0.838,0.937,0.866
41,K-80,245.8,581.4,394.1433,396.1498,146.0580,1150.063645,1,1,Tier 2,...,0.846,0.999,0.981,0.909,0.937,0.967,1.045,0.981,0.955,0.897
46,K-91,272.6,623.4,394.1433,396.1496,146.0581,1303.804362,1,1,Tier 2,...,0.848,0.992,1.006,0.938,1.004,1.023,0.992,0.958,1.020,0.939
51,K-112,314.4,687.8,322.1223,324.1288,74.0371,1735.581226,1,1,Tier 2,...,0.854,0.947,1.023,0.962,0.995,0.963,0.975,0.980,1.011,0.902
55,K-121,332.3,717.0,376.1326,378.1387,128.0474,1021.290947,1,1,Tier 2,...,1.144,1.072,0.975,1.037,1.127,1.171,1.062,1.034,1.026,0.984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,A-261,366.9,740.0,422.1749,424.1812,188.1166,1218.670033,1,1,Tier 2,...,2.546,1.581,3.585,1.077,1.112,1.040,1.057,1.001,1.012,1.049
776,A-276,396.5,824.5,335.1430,337.1494,101.0847,21583.925319,1,1,Tier 2,...,1.084,0.677,0.946,1.092,1.127,1.080,1.053,1.041,1.047,1.077
789,A-296,436.6,921.9,376.0856,378.0916,142.0273,1151.586552,1,1,Tier 2,...,0.362,0.159,0.565,1.088,1.100,1.068,1.059,1.039,1.067,1.050
801,A-321,467.3,1009.1,376.0854,378.0918,142.0271,1087.320634,1,1,Tier 2,...,0.561,2.377,0.736,1.028,1.105,1.071,1.059,1.004,0.988,1.090


### Extract the specified columns and transpose the DataFrame

In [73]:
# Define the columns to extract
columns = ['Normalized RT (s)', 'm/z_light', 'm/z_heavy', 'Neutral Mass (Da)', 'Intensity',
           'DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6', 'DR.7', 'No DR', 'DR.8',
           'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12', 'DR.13', 'DR.14', 'DR.15', 'DR.16',
           'No DR.2', 'DR.17', 'DR.18', 'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21',
           'DR.22', 'DR.23', 'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
           'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15', 'No DR.16',
           'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21', 'No DR.22', 'No DR.23',
           'No DR.24', 'No DR.25', 'No DR.26', 'No DR.27', 'No DR.28', 'No DR.29']

In [74]:
# Extract the specified columns
df_selected = df[columns]

In [75]:
# Iterate over each compound name
for compound in df_selected.columns:
    # Extract the data for the compound
    compound_data = df_selected[compound]
    # Create a new row with the compound name as the 'Sample ID' and assign the data to the corresponding columns
    new_row = pd.DataFrame([[compound] + list(compound_data)], columns=['Sample ID'] + list(df['Compound.1']))
    # Append the new row to the transposed DataFrame
    df_transposed = df_transposed.append(new_row, ignore_index=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [48]:
# Print the resulting DataFrame
df_transposed.columns

Index(['Sample ID',         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
       ...
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan],
      dtype='object', length=823)

In [44]:
# Create a new DataFrame for transposed data
df_transposed = pd.DataFrame(columns=['Sample ID'] + list(df['Compound.1']))

### Rename the columns to match the desired format

In [13]:
column_names = 